## A minimalist example for recovering sparse graphs using `mGLAD`

Fitting meta-GLAD on a batch of erdos-renyi random sparse graphs with samples obtained from a corresponding multivariate Gaussian distribution  

### About `mGLAD` 
A meta learning based approach to recover sparse graphs. This work proposes `mGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key Benefits & features:  
- It is a fast alternative to solving the Graphical Lasso problem as
    - GPU based acceleration can be leveraged
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization).     
- mGLAD automatically learns the sparsity related regularization parameters. This gives an added benefit to mGLAD over other graphical lasso solvers.  
- The meta loss is the logdet objective of the graphical lasso `1/B(-1*log|theta|+ <S, theta>)`, where `B=batch_size, S=input covariance matrix, theta=predicted precision matrix`.   

In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3/envs/dagM


In [2]:
import torch
torch.__version__

'1.9.0'

### Create Synthetic data

In [3]:
from scripts import main
# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = main.get_data(
    num_nodes=10, 
    sparsity=0.2, 
    num_samples=500, 
    batch_size=2
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (2, 10, 10), Samples (2, 500, 10)


### Running the meta-GLAD algorithm
Until Convergence:  

    1. Initialize learnable `GLAD` parameters
    2. Run the GLAD model
    3. Get the meta-loss
    4. Backprop

Possible reasons if `mGLAD` does not converge: 

    1. Please re-run. This will run the optimization with different initializations  
    2. Lower the learning rate  
    3. Change the INIT_DIAG=0/1 in the `GLAD` model parameters

In [4]:
pred_theta = main.run_mGLAD(Xb, true_theta, EPOCHS=250, VERBOSE=True)

epoch:0/250 loss:15.714613914489746
epoch:25/250 loss:15.55267333984375
epoch:50/250 loss:15.445390701293945
epoch:75/250 loss:14.959495544433594
epoch:100/250 loss:12.04627513885498
epoch:125/250 loss:10.557657241821289
epoch:150/250 loss:10.022600173950195
epoch:175/250 loss:9.71489429473877
epoch:200/250 loss:9.517026901245117
epoch:225/250 loss:9.384122848510742
Batch:0 - {'FDR': 0.0, 'TPR': 1.0, 'FPR': 0.0, 'SHD': 0, 'nnzTrue': 9, 'nnzPred': 9, 'precsion': 1.0, 'recall': 1.0, 'Fbeta': 1.0, 'aupr': 1.0, 'auc': 1.0}
Batch:1 - {'FDR': 0.0, 'TPR': 1.0, 'FPR': 0.0, 'SHD': 0, 'nnzTrue': 6, 'nnzPred': 6, 'precsion': 1.0, 'recall': 1.0, 'Fbeta': 1.0, 'aupr': 1.0, 'auc': 1.0}
